<a href="https://colab.research.google.com/github/m-manuelmussa/Molecular-Docking/blob/main/Pr%C3%A9_processamento_de_estruturas_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Pré-processamento de estruturas 3D**

*@Micliete_Mussa*

# **PIPELINE**
1. Importação de frameworks necessários
2. Execução da requisição e obtenção da resposta sobre as estruturas pdb a partir de UniProt
3. Selecção de estruturas a partir de critérios de interesse
4. Remoção de clones de estruturas pdb
5. Visualização da DataFrame de estruturas pré-processadas
6. Montagem da pasta na Drive**
7. Download das estruturas pré-processadas

### **1. Importação de frameworks necessários**

In [ ]:
import requests
from google.colab import drive
import os
import time
import pandas as pd

### **2. Execução da requisição e obtenção da resposta sobre as estruturas pdb a partir de UniProt**

In [ ]:
uniprot_id = "Q13936"  # Código UNIPROT substituível, outro (P35439, P22756)
url = f"https://www.ebi.ac.uk/pdbe/api/mappings/best_structures/{uniprot_id}"

try:
    response = requests.get(url, timeout=10).json()
except Exception as e:
    print(f"Erro ao buscar dados para UniProt {uniprot_id}: {e}")
    response = {}

In [ ]:
# Contar estruturas
if uniprot_id in response:
    total = len(response[uniprot_id])
print(f"Estruturas Disponíveis: {total}")

Estruturas Disponíveis: 50


### **3. Selecção de estruturas a partir de critérios de interesse**

In [ ]:
if uniprot_id in response:
    # Filtrar pela resolução ≤ 2.5 Å
    filtered = [
        entry for entry in response[uniprot_id]
        if entry.get("resolution") is not None and entry["resolution"] <= 2.5
    ]

### **4. Remoção de clones de estruturas pdb**

In [ ]:
    # Remover PDBs repetidos
    unique_pdbs = {}
    for entry in filtered:
        pdb_id = entry["pdb_id"]
        if pdb_id not in unique_pdbs:
            unique_pdbs[pdb_id] = entry

    filtered = list(unique_pdbs.values())

In [ ]:

total = len(filtered)
print(f"Estruturas Disponíveis: {total}")

Estruturas Disponíveis: 14


### **5. Visualização da DataFrame de estruturas pré-processadas**

In [ ]:
if filtered:
    print(f"Estruturas com resolução ≤ 2.5 Å para {uniprot_id} (sem duplicatas):")
    # Transformar em DataFrame para visualização
    df = pd.DataFrame(filtered)
    display(df[["pdb_id", "resolution", "experimental_method"]].sort_values("resolution"))

Estruturas com resolução ≤ 2.5 Å para Q13936 (sem duplicatas):


,pdb_id,resolution,experimental_method
9,2f3y,1.45,X-ray diffraction
10,2f3z,1.60,X-ray diffraction
1,6dad,1.65,X-ray diffraction
2,6u3a,1.65,X-ray diffraction
3,6u3b,1.70,X-ray diffraction
11,5v2q,1.70,X-ray diffraction
4,6u3d,1.75,X-ray diffraction
6,6dae,2.00,X-ray diffraction
13,1t0j,2.00,X-ray diffraction
12,5v2p,2.00,X-ray diffraction


In [ ]:
df.to_excel("Estruturas seleccionadas.xlsx", index=False)

### **6. Montagem da pasta na Drive**

In [ ]:
# Montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Montagem da pasta com UNIPROT ID
pasta = os.path.join("/content/drive/MyDrive", uniprot_id)
os.makedirs(pasta, exist_ok=True)

### **7. Download das estruturas pré-processadas**

In [ ]:
# Baixar os PDBs
for entry in filtered:
    pdb_id = entry["pdb_id"]
    resolucao = entry.get("resolution", "N/A")
    metodo = entry.get("experimental_method", "N/A")
    url_pdb = f"https://www.ebi.ac.uk/pdbe/entry-files/pdb{pdb_id}.ent"
    nome_arquivo = os.path.join(pasta, f"{pdb_id}.pdb")

    try:
        r = requests.get(url_pdb, timeout=10)
        if r.status_code == 200:
            with open(nome_arquivo, "wb") as f:
                f.write(r.content)
            print(f"Baixado {pdb_id} ({resolucao} Å, {metodo})")
        else:
            print(f"Não foi possível baixar {pdb_id} — Status: {r.status_code}")
    except Exception as e:
        print(f"Erro ao baixar {pdb_id}: {e}")

    time.sleep(0.05)  # evitar sobrecarga na API

Baixado 3g43 (2.1 Å, X-ray diffraction)
Baixado 6dad (1.65 Å, X-ray diffraction)
Baixado 6u3a (1.65 Å, X-ray diffraction)
Baixado 6u3b (1.7 Å, X-ray diffraction)
Baixado 6u3d (1.75 Å, X-ray diffraction)
Baixado 2be6 (2.0 Å, X-ray diffraction)
Baixado 6dae (2.0 Å, X-ray diffraction)
Baixado 6daf (2.4 Å, X-ray diffraction)
Baixado 6u39 (2.4 Å, X-ray diffraction)
Baixado 2f3y (1.45 Å, X-ray diffraction)
Baixado 2f3z (1.6 Å, X-ray diffraction)
Baixado 5v2q (1.7 Å, X-ray diffraction)
Baixado 5v2p (2.0 Å, X-ray diffraction)
Baixado 1t0j (2.0 Å, X-ray diffraction)


# **FIM**